```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Views](#views)**
3. **[Stored Procedures](#stored-procedures)**
4. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb",
        "src/DataFactory/Statics/Opsplitsen_Number_Municipality.ipynb"
    ]
}
```

# **TABLES**

---


### STAGING

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[NICE_HOSPITAL]'))
CREATE TABLE [VWSSTAGE].[NICE_HOSPITAL](
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[VERSION] VARCHAR(100) NULL,
	[DATE_OF_REPORT] VARCHAR(100) NULL,
	[DATE_OF_STATISTICS] VARCHAR(100) NULL,
	[MUNICIPALITY_CODE] VARCHAR(100) NULL,
	[MUNICIPALITY_NAME] VARCHAR(100) NULL,
	[SECURITY_REGION_CODE] VARCHAR(100) NULL,
	[SECURITY_REGION_NAME] VARCHAR(100) NULL,
	[HOSPITAL_ADMISSION_NOTIFICATION] VARCHAR(100) NULL,
	[HOSPITAL_ADMISSION] VARCHAR(100) NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

### INTERMEDIATES

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[NICE_HOSPITAL]'))
CREATE TABLE [VWSINTER].[NICE_HOSPITAL](
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[MUNICIPALITY_CODE] VARCHAR(100) NULL,
	[MUNICIPALITY_NAME] VARCHAR(100) NULL,
	[SECURITY_REGION_CODE] VARCHAR(100) NULL,
	[SECURITY_REGION_NAME] VARCHAR(100) NULL,
	[REPORTED] INT NULL,
	[HOSPITALIZED] INT NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

### DESTATIONS

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[NICE_HOSPITAL_GM]'))
BEGIN
    CREATE TABLE [VWSDEST].[NICE_HOSPITAL_GM] (
	    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
        [DATE_OF_STATISTICS]         DATETIME        NULL,
        [GM_CODE]                    VARCHAR (100)   NULL,
        [GM_NAME]                    VARCHAR (100)   NULL,
        [REPORTED]                   INT             NULL,
        [HOSPITALIZED]               INT             NULL,
        [HOSPITALIZED_3D_AVG]        DECIMAL (16, 2) NULL,
        [REPORTED_3D_AVG]            DECIMAL (16, 2) NULL,
        [DATE_LAST_INSERTED]         DATETIME        DEFAULT (getdate()) NULL,
        [REPORTED_PER_MIL_LAST_WEEK] INT             NULL,
        [WEEK_START]                 DATETIME        NULL,
        [HOSPITALIZED_7D_AVG]        DECIMAL (16, 2) NULL,
        [WEEK_START_LAG]             DATETIME        NULL,
        [HOSPITALIZED_7D_AVG_LAG]    DECIMAL (16, 2) NULL,
        [REPORTED_7D_AVG]            DECIMAL (16, 2) NULL,
        [REPORTED_7D_AVG_LAG]        DECIMAL (16, 2) NULL,
        [HOSPITALIZED_7D_AVG_CUTOFF] DECIMAL (16, 2) NULL,
        [HOSPITALIZED_PER_100000]    DECIMAL (16, 2) NULL
    );
END
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
    WHERE [NAME] IN (N'HOSPITALIZED_PER_100000')
    AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[NICE_HOSPITAL_GM]'))
        BEGIN
            ALTER TABLE [VWSDEST].[NICE_HOSPITAL_GM] 
            ADD 
                [HOSPITALIZED_PER_100000] DECIMAL (16, 2) NULL;
        END
END

GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_GM_DLI_2')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_GM_DLI_2]
    ON [VWSDEST].[NICE_HOSPITAL_GM]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_STATISTICS], [GM_CODE], [GM_NAME], [REPORTED], [HOSPITALIZED]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_GM_DLI_3')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_GM_DLI_3]
    ON [VWSDEST].[NICE_HOSPITAL_GM]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_STATISTICS], [GM_CODE], [GM_NAME], [REPORTED], [HOSPITALIZED], [HOSPITALIZED_7D_AVG]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_GM_DLI_DS')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_GM_DLI_DS]
    ON [VWSDEST].[NICE_HOSPITAL_GM]([DATE_LAST_INSERTED] ASC, [DATE_OF_STATISTICS] ASC)
    INCLUDE([GM_CODE], [GM_NAME], [HOSPITALIZED_3D_AVG]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_GM_DS')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_GM_DS]
    ON [VWSDEST].[NICE_HOSPITAL_GM]([DATE_OF_STATISTICS] ASC)
    INCLUDE([GM_CODE], [GM_NAME], [HOSPITALIZED_3D_AVG], [DATE_LAST_INSERTED]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_INTER_NICE_HOSPITAL_GM_DLI_DS')
CREATE NONCLUSTERED INDEX [IX_INTER_NICE_HOSPITAL_GM_DLI_DS]
    ON [VWSDEST].[NICE_HOSPITAL_GM]([DATE_LAST_INSERTED] ASC, [DATE_OF_STATISTICS] ASC)
    INCLUDE([GM_CODE], [REPORTED], [HOSPITALIZED_3D_AVG]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_GM_DEST')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_GM_DEST]
    ON [VWSDEST].[NICE_HOSPITAL_GM]([DATE_OF_STATISTICS] ASC, [DATE_LAST_INSERTED] ASC, [GM_CODE] ASC)
    INCLUDE([HOSPITALIZED], [HOSPITALIZED_3D_AVG]);





In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[NICE_HOSPITAL_VR]'))
BEGIN
    CREATE TABLE [VWSDEST].[NICE_HOSPITAL_VR] (
        [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
        [DATE_OF_STATISTICS]         DATETIME        NULL,
        [VR_CODE]                    VARCHAR (100)   NULL,
        [VR_NAME]                    VARCHAR (100)   NULL,
        [REPORTED]                   INT             NULL,
        [HOSPITALIZED]               INT             NULL,
        [HOSPITALIZED_3D_AVG]        DECIMAL (16, 2) NULL,
        [REPORTED_3D_AVG]            DECIMAL (16, 2) NULL,
        [HOSPITALIZED_CUMULATIVE]    INT             NULL,
        [REPORTED_CUMULATIVE]        INT             NULL,
        [DATE_LAST_INSERTED]         DATETIME        DEFAULT (getdate()) NULL,
        [REPORTED_PER_MIL_LAST_WEEK] INT             NULL,
        [WEEK_START]                 DATETIME        NULL,
        [HOSPITALIZED_7D_AVG]        DECIMAL (16, 2) NULL,
        [WEEK_START_LAG]             DATETIME        NULL,
        [HOSPITALIZED_7D_AVG_LAG]    DECIMAL (16, 2) NULL,
        [REPORTED_7D_AVG]            DECIMAL (16, 2) NULL,
        [REPORTED_7D_AVG_LAG]        DECIMAL (16, 2) NULL,
        [HOSPITALIZED_7D_AVG_CUTOFF] DECIMAL (16, 2) NULL,
        [HOSPITALIZED_PER_100000]    DECIMAL (16, 2) NULL
    );
END
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
    WHERE [NAME] IN (N'HOSPITALIZED_PER_100000')
    AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[NICE_HOSPITAL_VR]'))
        BEGIN
            ALTER TABLE [VWSDEST].[NICE_HOSPITAL_VR] 
            ADD 
                [HOSPITALIZED_PER_100000] DECIMAL (16, 2) NULL;
        END
END

GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_VR')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_VR]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_LAST_INSERTED] ASC, [DATE_OF_STATISTICS] ASC)
    INCLUDE([VR_CODE]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_VR_AVG')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_VR_AVG]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_LAST_INSERTED] ASC, [DATE_OF_STATISTICS] ASC)
    INCLUDE([VR_CODE], [HOSPITALIZED_3D_AVG]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_VR_DLI')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_VR_DLI]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_STATISTICS], [VR_CODE], [HOSPITALIZED], [HOSPITALIZED_3D_AVG], [HOSPITALIZED_CUMULATIVE]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_NICE_HOSPITAL_VR_DLI_DS')
CREATE NONCLUSTERED INDEX [IX_DEST_NICE_HOSPITAL_VR_DLI_DS]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_LAST_INSERTED] ASC, [DATE_OF_STATISTICS] ASC)
    INCLUDE([VR_CODE], [HOSPITALIZED_CUMULATIVE]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_NL_DEST')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_NL_DEST]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_OF_STATISTICS] ASC, [DATE_LAST_INSERTED] ASC)
    INCLUDE([HOSPITALIZED], [HOSPITALIZED_3D_AVG]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_VR_DATE_LAST_INSERTED')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_VR_DATE_LAST_INSERTED]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_STATISTICS], [VR_CODE], [REPORTED]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_VR_DEST')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_VR_DEST]
    ON [VWSDEST].[NICE_HOSPITAL_VR]([DATE_OF_STATISTICS] ASC, [DATE_LAST_INSERTED] ASC, [VR_CODE] ASC)
    INCLUDE([HOSPITALIZED], [HOSPITALIZED_3D_AVG], [HOSPITALIZED_CUMULATIVE]);






In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[NICE_HOSPITAL_NL]'))
CREATE TABLE [VWSDEST].[NICE_HOSPITAL_NL] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [DATE_OF_STATISTICS]         DATETIME        NULL,
    [REPORTED]                   INT             NULL,
    [HOSPITALIZED]               INT             NULL,
    [HOSPITALIZED_3D_AVG]        DECIMAL (16, 2) NULL,
    [REPORTED_3D_AVG]            DECIMAL (16, 2) NULL,
    [DATE_LAST_INSERTED]         DATETIME        DEFAULT (getdate()) NULL,
    [REPORTED_PER_MIL_LAST_WEEK] INT             NULL,
    [WEEK_START]                 DATETIME        NULL,
    [HOSPITALIZED_7D_AVG]        DECIMAL (16, 2) NULL,
    [WEEK_START_LAG]             DATETIME        NULL,
    [HOSPITALIZED_7D_AVG_LAG]    DECIMAL (16, 2) NULL,
    [REPORTED_7D_AVG]            DECIMAL (16, 2) NULL,
    [REPORTED_7D_AVG_LAG]        DECIMAL (16, 2) NULL,
    [HOSPITALIZED_7D_AVG_CUTOFF] DECIMAL (16, 2) NULL
);
GO

GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_NL_DATE_LAST_INSERTED_10')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_NL_DATE_LAST_INSERTED_10]
    ON [VWSDEST].[NICE_HOSPITAL_NL]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_STATISTICS], [REPORTED], [HOSPITALIZED]);



In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[NICE_HOSPITAL_GM_BASE]'))
CREATE TABLE [VWSDEST].[NICE_HOSPITAL_GM_BASE] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [DATE_OF_REPORT]          DATETIME        NULL,
    [NEW_DATE_OF_REPORT_UNIX] BIGINT          NULL,
    [GMCODE]                  VARCHAR (100)   NULL,
    [OLD_DATE_OF_REPORT_UNIX] BIGINT          NULL,
    [OLD_VALUE]               DECIMAL (16, 2) NULL,
    [DIFFERENCE]              DECIMAL (16, 2) NULL,
    [DATE_LAST_INSERTED]      DATETIME        DEFAULT (getdate()) NULL,
    [OLD_VALUE_REPORTED]      INT             NULL,
    [DIFFERENCE_REPORTED]     INT             NULL
);
GO


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_GM_BASE')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_GM_BASE]
    ON [VWSDEST].[NICE_HOSPITAL_GM_BASE]([DATE_OF_REPORT] ASC, [DATE_LAST_INSERTED] ASC, [GMCODE] ASC)
    INCLUDE([DIFFERENCE], [OLD_VALUE]);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_GM_BASE_GMCODE')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_GM_BASE_GMCODE]
    ON [VWSDEST].[NICE_HOSPITAL_GM_BASE]([GMCODE] ASC);


GO
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_NICE_HOSPITAL_GM_BASE_GMCODE_DATE_LAST_INSERTED')
CREATE NONCLUSTERED INDEX [IX_NICE_HOSPITAL_GM_BASE_GMCODE_DATE_LAST_INSERTED]
    ON [VWSDEST].[NICE_HOSPITAL_GM_BASE]([GMCODE] ASC, [DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_REPORT], [NEW_DATE_OF_REPORT_UNIX], [OLD_DATE_OF_REPORT_UNIX], [OLD_VALUE_REPORTED], [DIFFERENCE_REPORTED]);



# **VIEWS**

---

### GM

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- JSON-KEY: hospital_nice PER GM_COLLECTION

CREATE OR ALTER VIEW [VWSDEST].[V_HOSPITAL_ADMISSIONS_GM_LAST_NICE] AS
SELECT
    DATE_UNIX,
    GMCODE,
    ADMISSIONS_ON_DATE_OF_ADMISSION,
    ADMISSIONS_ON_DATE_OF_REPORTING,
    ADMISSIONS_ON_DATE_OF_ADMISSION_PER_100000,
    DATE_OF_INSERTION_UNIX

FROM(
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],
    [GM_CODE] AS [GMCODE],
    [DBO].[NO_NEGATIVE_NUMBER_F](HOSPITALIZED) AS [ADMISSIONS_ON_DATE_OF_ADMISSION],
    [DBO].[NO_NEGATIVE_NUMBER_F](REPORTED) AS [ADMISSIONS_ON_DATE_OF_REPORTING],
    CAST([HOSPITALIZED_PER_100000] AS NUMERIC(10,1)) AS [ADMISSIONS_ON_DATE_OF_ADMISSION_PER_100000],
    [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX],
    RANK() OVER (PARTITION BY [GM_CODE] ORDER BY [DATE_OF_STATISTICS] DESC) AS [RANKING]
FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK)
WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK))
AND [WEEK_START] <> [WEEK_START_LAG]) AS T1
WHERE T1.[RANKING] =1
AND [GMCODE] !=''
GO

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_NICE_HOSPITAL_CHOROPLETH_GM_COLLECTION] AS
WITH
    CTE
    AS (
        SELECT DISTINCT
            MAX([DATE_OF_STATISTICS]) AS [DATE_OF_STATISTICS]
        FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK))
            AND [HOSPITALIZED_7D_AVG_CUTOFF] IS NOT NULL
    )
    SELECT
        [dbo].[CONVERT_DATETIME_TO_UNIX](T1.[DATE_OF_STATISTICS]) AS [DATE_UNIX],
        [GM_CODE] AS [GMCODE],
        [DBO].[NO_NEGATIVE_NUMBER_F]([HOSPITALIZED]) AS [ADMISSIONS_ON_DATE_OF_ADMISSION],
        [DBO].[NO_NEGATIVE_NUMBER_F]([REPORTED]) AS [ADMISSIONS_ON_DATE_OF_REPORTING],
        CAST([HOSPITALIZED_PER_100000] AS NUMERIC(10,1)) AS [ADMISSIONS_ON_DATE_OF_ADMISSION_PER_100000],
        [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
    FROM [VWSDEST].[NICE_HOSPITAL_GM] T1 WITH(NOLOCK)
        INNER JOIN CTE T2 ON T2.[DATE_OF_STATISTICS] = T1.[DATE_OF_STATISTICS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK))
        AND [WEEK_START] <> [WEEK_START_LAG]
        AND TRIM([GM_CODE]) != ''
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

-- JSON-KEY: hospital_nice PER GM

CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_GM_NICE AS
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],
    GM_CODE AS GMCODE,
    GM_NAME AS MUNICIPALITY_NAME,
    DBO.NO_NEGATIVE_NUMBER_I(HOSPITALIZED) AS ADMISSIONS_ON_DATE_OF_ADMISSION,
    HOSPITALIZED_7D_AVG_CUTOFF AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE,
    CAST(ROUND(HOSPITALIZED_7D_AVG_CUTOFF,0) AS INTEGER) AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED,
    DBO.NO_NEGATIVE_NUMBER_I(REPORTED) AS ADMISSIONS_ON_DATE_OF_REPORTING,
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_GM WITH(NOLOCK)
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_GM WITH(NOLOCK));
GO

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_NICE_HOSPITAL_CHOROPLETH_GM] AS
WITH
    CTE
    AS (
        SELECT DISTINCT
            MAX([DATE_OF_STATISTICS]) AS [DATE_OF_STATISTICS]
        FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NICE_HOSPITAL_GM] WITH(NOLOCK))
            AND [HOSPITALIZED_7D_AVG_CUTOFF] IS NOT NULL
    )
    SELECT
        [dbo].[CONVERT_DATETIME_TO_UNIX](T1.[DATE_OF_STATISTICS]) AS [DATE_UNIX],
        [GM_CODE] AS [GMCODE],
        [GM_NAME] AS [MUNICIPALITY_NAME],
        DBO.NO_NEGATIVE_NUMBER_I(HOSPITALIZED) AS [ADMISSIONS_ON_DATE_OF_ADMISSION],
        DBO.NO_NEGATIVE_NUMBER_I(REPORTED) AS [ADMISSIONS_ON_DATE_OF_REPORTING],
        dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX]
    FROM VWSDEST.NICE_HOSPITAL_GM T1 WITH(NOLOCK)
        INNER JOIN CTE T2 ON T2.[DATE_OF_STATISTICS] = T1.[DATE_OF_STATISTICS]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_GM WITH(NOLOCK))
        AND [WEEK_START] <> [WEEK_START_LAG]
        AND TRIM([GM_CODE]) != ''
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

--THIS VIEW IS NOT USED FOR PROTOS
CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_GM_REL_NICE AS
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([WEEK_START])          AS [DATE_START_UNIX],
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS])  AS [DATE_END_UNIX],
    GM_CODE AS GMCODE,
	REPORTED_PER_MIL_LAST_WEEK,
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_GM
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_GM)
        AND WEEK_START IS NOT NULL; --Only records with 7 days of history

### VR

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor  more information.
/****** Object:  View [VWSDEST].[V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_MOVING_AVG_PER_REGION_NICE]    Script Date: 12-11-2020 15:14:55 ******/

--Previously [VWSDEST].[V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_MOVING_AVG_PER_REGION_NICE] was used
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_VR] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		VR_CODE,
		CAST(MAX(DATE_OF_STATISTICS) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM VWSDEST.NICE_HOSPITAL_VR
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_VR)
	GROUP BY VR_CODE
),

BASE_CTE AS (
SELECT 
	DATE_OF_STATISTICS AS DATE_OF_REPORT,
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [NEW_DATE_OF_REPORT_UNIX],
	VR_CODE,
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [OLD_DATE_OF_REPORT_UNIX],
    REPORTED,
	REPORTED_7D_AVG AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_F(REPORTED) -
		DBO.NO_NEGATIVE_NUMBER_F(REPORTED_7D_AVG) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_VR
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_VR)
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,T1.VR_CODE AS VRCODE
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CAST(round([OLD_VALUE], 0) as INT) AS [OLD_VALUE]
	  ,DBO.NO_NEGATIVE_NUMBER_F(REPORTED) -
		CAST(round([OLD_VALUE], 0) as INT) AS [DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
	ON T1.[VR_CODE] = T2.[VR_CODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
;

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor  more information.
/****** Object:  View [VWSDEST].[V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_MOVING_AVG_PER_REGION_NICE]    Script Date: 12-11-2020 15:14:55 ******/
--THIS VIEW IS NOT USED FOR PROTOS
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_MOVING_AVG_PER_REGION_NICE] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		VR_CODE,
		CAST(MAX(DATE_OF_STATISTICS) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM VWSDEST.NICE_HOSPITAL_VR
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_VR)
	GROUP BY VR_CODE
),

BASE_CTE AS (
SELECT 
	DATE_OF_STATISTICS AS DATE_OF_REPORT,
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [NEW_DATE_OF_REPORT_UNIX],
	VR_CODE,
	LAG(dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]), 1, NULL) OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_LAST_INSERTED], DATE_OF_STATISTICS) AS [OLD_DATE_OF_REPORT_UNIX],
    LAG(DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED_3D_AVG), 1, NULL) OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_STATISTICS]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED_3D_AVG) -
		LAG(DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED_3D_AVG), 1, NULL) OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_STATISTICS]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_VR
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_VR)
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,T1.VR_CODE AS VRCODE
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,OLD_VALUE
	  ,[DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
	ON T1.[VR_CODE] = T2.[VR_CODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
;

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor  more information.
/****** Object:  View [VWSDEST].[V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_TOTAL_COUNTS_PER_REGION_NICE]    Script Date: 12-11-2020 15:14:55 ******/
--THIS VIEW IS NOT USED FOR PROTOS
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_TOTAL_COUNTS_PER_REGION_NICE] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		VR_CODE,
		CAST(MAX(DATE_OF_STATISTICS) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM VWSDEST.NICE_HOSPITAL_VR
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_VR)
	GROUP BY VR_CODE
),

BASE_CTE AS (
SELECT 
	DATE_OF_STATISTICS AS DATE_OF_REPORT,
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [NEW_DATE_OF_REPORT_UNIX],
	VR_CODE,
	LAG(dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]), 1, NULL) OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_LAST_INSERTED], DATE_OF_STATISTICS) AS [OLD_DATE_OF_REPORT_UNIX],
    LAG(DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED_CUMULATIVE), 1, NULL) OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_LAST_INSERTED], DATE_OF_STATISTICS) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED_CUMULATIVE) -
		LAG(DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED_CUMULATIVE), 1, NULL) OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_LAST_INSERTED], DATE_OF_STATISTICS) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_VR
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_VR)
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,T1.VR_CODE AS VRCODE
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,OLD_VALUE
	  ,[DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
	ON T1.[VR_CODE] = T2.[VR_CODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
;

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_VR_NICE AS
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],
    VR_CODE AS VRCODE,
    VR_NAME AS VRNAME,
    DBO.NO_NEGATIVE_NUMBER_I(HOSPITALIZED) AS ADMISSIONS_ON_DATE_OF_ADMISSION,
    HOSPITALIZED_7D_AVG_CUTOFF AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE,
    CAST(ROUND(HOSPITALIZED_7D_AVG_CUTOFF,0) AS INTEGER) AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED,
    DBO.NO_NEGATIVE_NUMBER_I(REPORTED) AS ADMISSIONS_ON_DATE_OF_REPORTING,
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_VR WITH(NOLOCK)
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_VR WITH(NOLOCK));
GO

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_NICE_HOSPITAL_CHOROPLETH_VR] AS
WITH
    CTE
    AS (
        SELECT DISTINCT
            MAX([DATE_OF_STATISTICS]) AS [DATE_OF_STATISTICS]
        FROM [VWSDEST].[NICE_HOSPITAL_VR] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NICE_HOSPITAL_VR] WITH(NOLOCK))
            AND [HOSPITALIZED_7D_AVG_CUTOFF] IS NOT NULL
    )
    SELECT
        [dbo].[CONVERT_DATETIME_TO_UNIX](T1.[DATE_OF_STATISTICS]) AS [DATE_UNIX],
        VR_CODE AS VRCODE,
        VR_NAME AS VRNAME,
        DBO.NO_NEGATIVE_NUMBER_I(HOSPITALIZED) AS ADMISSIONS_ON_DATE_OF_ADMISSION,
        DBO.NO_NEGATIVE_NUMBER_I(REPORTED) AS ADMISSIONS_ON_DATE_OF_REPORTING,
        dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
    FROM VWSDEST.NICE_HOSPITAL_VR T1 WITH(NOLOCK)
        INNER JOIN CTE T2 ON T2.[DATE_OF_STATISTICS] = T1.[DATE_OF_STATISTICS]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_VR  WITH(NOLOCK))
        AND [VR_CODE] != ''
GO


In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_VR_REL_NICE AS
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([WEEK_START])          AS [DATE_START_UNIX],
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS])  AS [DATE_END_UNIX],
    VR_CODE AS VRCODE,
    REPORTED_PER_MIL_LAST_WEEK                              AS ADMISSIONS_PER_1M,
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_VR
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_VR)
        AND WEEK_START IS NOT NULL; --Only records with 7 days of history

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- nice_hospital FOR VR_COLLECTION

CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_VR_LAST_NICE AS 
SELECT [DATE_OF_REPORT_UNIX] AS [DATE_UNIX],
       [VRCODE], 
       [ADMISSIONS_ON_DATE_OF_ADMISSION],
       [ADMISSIONS_ON_DATE_OF_REPORTING],
        [ADMISSIONS_ON_DATE_OF_ADMISSION_PER_100000],
       [DATE_OF_INSERTION_UNIX]
FROM (
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_OF_REPORT_UNIX],
    [VR_CODE] as [VRCODE],
    [DBO].[NO_NEGATIVE_NUMBER_F](HOSPITALIZED) AS [ADMISSIONS_ON_DATE_OF_ADMISSION],
    [DBO].[NO_NEGATIVE_NUMBER_F](REPORTED) AS [ADMISSIONS_ON_DATE_OF_REPORTING],
    CAST([HOSPITALIZED_PER_100000] AS NUMERIC(10,1)) AS [ADMISSIONS_ON_DATE_OF_ADMISSION_PER_100000],
    [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX],
    RANK() OVER (PARTITION BY [VR_CODE] ORDER BY [DATE_OF_STATISTICS] DESC) [RANKING]
FROM [VWSDEST].[NICE_HOSPITAL_VR] WITH(NOLOCK)
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_VR WITH(NOLOCK))
AND VR_CODE IS NOT NULL
AND TRIM(VR_CODE) != '') T1
WHERE T1.RANKING = 1
GO

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_NICE_HOSPITAL_CHOROPLETH_VR_COLLECTION] AS 
WITH
    CTE
    AS (
        SELECT DISTINCT
            MAX([DATE_OF_STATISTICS]) AS [DATE_OF_STATISTICS]
        FROM [VWSDEST].[NICE_HOSPITAL_VR] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NICE_HOSPITAL_VR] WITH(NOLOCK))
            AND [HOSPITALIZED_7D_AVG_CUTOFF] IS NOT NULL
    )
    SELECT
        [DBO].[CONVERT_DATETIME_TO_UNIX](T1.[DATE_OF_STATISTICS]) AS [DATE_UNIX],
        VR_CODE AS VRCODE,
        DBO.NO_NEGATIVE_NUMBER_F(HOSPITALIZED) AS [ADMISSIONS_ON_DATE_OF_ADMISSION],
        DBO.NO_NEGATIVE_NUMBER_F(REPORTED) AS [ADMISSIONS_ON_DATE_OF_REPORTING],
        CAST([HOSPITALIZED_PER_100000] AS NUMERIC(10,1)) AS [ADMISSIONS_ON_DATE_OF_ADMISSION_PER_100000],
        DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX]
    FROM [VWSDEST].[NICE_HOSPITAL_VR] T1 WITH(NOLOCK)
        INNER JOIN CTE T2 ON T2.[DATE_OF_STATISTICS] = T1.[DATE_OF_STATISTICS]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_VR WITH(NOLOCK))
    AND VR_CODE IS NOT NULL AND TRIM(VR_CODE) != ''
GO

### NL

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor  more information.
/****** Object:  View [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_MA__MOVING_AVERAGE_HOSPITAL_NICE]    Script Date: 15-12-2020 14:54:38 ******/
--THIS VIEW IS NOT USED FOR PROTOS
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_MA__MOVING_AVERAGE_HOSPITAL_NICE] AS
WITH LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_STATISTICS) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM VWSDEST.NICE_HOSPITAL_NL
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_NL)
),

BASE_CTE AS (

SELECT 
	DATE_OF_STATISTICS AS DATE_OF_REPORT,
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [NEW_DATE_OF_REPORT_UNIX],
	LAG(dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_STATISTICS]) AS [OLD_DATE_OF_REPORT_UNIX],
    LAG(DBO.NO_NEGATIVE_NUMBER_F([HOSPITALIZED_3D_AVG]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_STATISTICS]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_F([HOSPITALIZED_3D_AVG]) -
		LAG(DBO.NO_NEGATIVE_NUMBER_F([HOSPITALIZED_3D_AVG]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_STATISTICS]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_NL
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_NL)
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,OLD_VALUE
	  ,[DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
/****** Object:  View [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_MA__MOVING_AVERAGE_HOSPITAL_NICE]    Script Date: 15-12-2020 14:54:38 ******/

--Previously [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_MA__MOVING_AVERAGE_HOSPITAL_NICE] was used
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_NL] AS
WITH LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_STATISTICS) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM VWSDEST.NICE_HOSPITAL_NL
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_NL)
),

BASE_CTE AS (

SELECT 
	DATE_OF_STATISTICS AS DATE_OF_REPORT,
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [NEW_DATE_OF_REPORT_UNIX],
	dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [OLD_DATE_OF_REPORT_UNIX],
    REPORTED_7D_AVG AS [OLD_VALUE],
	[REPORTED]
FROM VWSDEST.NICE_HOSPITAL_NL
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM VWSDEST.NICE_HOSPITAL_NL)
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX					AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX					AS OLD_DATE_UNIX
	  ,CAST(round(OLD_VALUE, 0) as INT)			AS [OLD_VALUE]
	  ,DBO.NO_NEGATIVE_NUMBER_F([REPORTED]) -
		CAST(round(OLD_VALUE, 0) as INT)		AS [DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- JSON-KEY: nice_hospital FOR NL.json

CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_NL_NICE AS
SELECT
    dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS DATE_UNIX
,   HOSPITALIZED AS ADMISSIONS_ON_DATE_OF_ADMISSION
,   HOSPITALIZED_7D_AVG_CUTOFF AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE
,   CAST(ROUND(HOSPITALIZED_7D_AVG_CUTOFF,0) AS INTEGER) AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED
,   REPORTED AS ADMISSIONS_ON_DATE_OF_REPORTING
,   dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)  AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_NL WITH(NOLOCK)
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_NL WITH(NOLOCK))
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.


--THIS VIEW IS NOT USED FOR PROTOS
CREATE OR ALTER VIEW VWSDEST.V_HOSPITAL_ADMISSIONS_NL_REL_NICE AS
SELECT
    [dbo].[CONVERT_DATETIME_TO_UNIX]([WEEK_START])          AS [DATE_START_UNIX],
    [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS])  AS [DATE_END_UNIX],
    REPORTED_PER_MIL_LAST_WEEK,
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.NICE_HOSPITAL_NL
  WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_NL)
        AND WEEK_START IS NOT NULL --Only records with 7 days of history

### GM_BASE

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor  more information.
--THIS VIEW IS NOT USED FOR PROTOS
CREATE OR ALTER VIEW  [VWSDEST].[V_DIFFERENCE_HOSPITAL_ADMISSIONS__MOVING_AVERAGE_HOSPITAL_NICE]
AS
SELECT
    DATE_OF_REPORT
    ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
    ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
    ,GMCODE
    ,OLD_VALUE
    ,[DIFFERENCE]
FROM VWSDEST.NICE_HOSPITAL_GM_BASE
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_GM_BASE)

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor  more information.

--Previously [VWSDEST].[V_DIFFERENCE_HOSPITAL_ADMISSIONS__MOVING_AVERAGE_HOSPITAL_NICE] was used
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_GM]
AS
SELECT
    DATE_OF_REPORT
    ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
    ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
    ,GMCODE
    ,CAST(OLD_VALUE AS INT) AS [OLD_VALUE]
    ,CAST(DIFFERENCE AS INT) AS [DIFFERENCE]
FROM VWSDEST.NICE_HOSPITAL_GM_BASE
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_GM_BASE)

# **STORE PROCEDURES**
---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  PROCEDURE [dbo].[SP_NICE_HOSPITAL_INTER]
 AS
 BEGIN
 
 -- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

 -- MAIN SELECT AND INSERT INTO STATEMENT. FILTERED VALUES ON MAX DATELASTINSERTED.
 -- MOVE INFECTIOUS DATA FROM STAGING TO INTERMEDIATE TABLE.
 INSERT INTO [VWSINTER].[NICE_HOSPITAL](
 	[DATE_OF_REPORT]
     ,[DATE_OF_STATISTICS]
     ,[MUNICIPALITY_CODE]
     ,[MUNICIPALITY_NAME]
     ,[SECURITY_REGION_CODE]
     ,[SECURITY_REGION_NAME]
     ,[REPORTED]
     ,[HOSPITALIZED]
     ,DATE_LAST_INSERTED
 )
     SELECT
         CAST(A.[DATE_OF_REPORT] AS [DATETIME]) AS [DATE_OF_REPORT]
         ,CAST(A.[DATE_OF_STATISTICS] AS [DATETIME]) AS [DATE_OF_STATISTICS]
         ,ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
         ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
         ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
         ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [REPORTED]
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [HOSPITALIZED]
         ,@DateTimeNow as Date_Last_Inserted
     FROM [VWSSTAGE].[NICE_HOSPITAL] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[NICE_HOSPITAL])
     GROUP BY ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
             ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
             ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
             ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
             ,A.[DATE_OF_REPORT]
             ,A.[DATE_OF_STATISTICS]

 INSERT INTO [VWSINTER].[NICE_HOSPITAL](
 	[DATE_OF_REPORT]
     ,[DATE_OF_STATISTICS]
     ,[MUNICIPALITY_CODE]
     ,[MUNICIPALITY_NAME]
     ,[SECURITY_REGION_CODE]
     ,[SECURITY_REGION_NAME]
     ,[REPORTED]
     ,[HOSPITALIZED]
     ,DATE_LAST_INSERTED
 )
     SELECT
         CAST(A.[DATE_OF_REPORT] AS [DATETIME]) AS [DATE_OF_REPORT]
         ,CAST(A.[DATE_OF_STATISTICS] AS [DATETIME]) AS [DATE_OF_STATISTICS]
         ,ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
         ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
         ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
         ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [REPORTED]
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [HOSPITALIZED]
         ,@DateTimeNow as Date_Last_Inserted
     FROM [VWSSTATIC].[NICE_HOSPITAL] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE CAST(A.DATE_LAST_INSERTED as DATE) = (SELECT MAX(CAST(DATE_LAST_INSERTED as DATE)) FROM [VWSSTATIC].[NICE_HOSPITAL])
     GROUP BY ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
             ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
             ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
             ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
             ,A.[DATE_OF_REPORT]
             ,A.[DATE_OF_STATISTICS]

 END;
 
GO

### INTERMEDIATE &rarr; DEST MAPPING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER PROCEDURE [dbo].[SP_NICE_HOSPITAL_GM]
AS
BEGIN
WITH BASE_CTE AS (
	SELECT
		DATE_OF_STATISTICS
	,	MUNICIPALITY_CODE
	,	SUM(HOSPITALIZED) AS HOSPITALIZED
	,	SUM(REPORTED) AS REPORTED
	FROM VWSINTER.NICE_HOSPITAL
	WHERE MUNICIPALITY_CODE!='NA'
	AND DATE_LAST_INSERTED=(SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.NICE_HOSPITAL)
	GROUP BY MUNICIPALITY_CODE, DATE_OF_STATISTICS
)

,SECOND_CTE AS (    
	SELECT
		DATE_OF_STATISTICS
	,	MUNICIPALITY_CODE
	,	GMNAAM
	,	HOSPITALIZED
	,	REPORTED
	,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as HOSPITALIZED_3D_AVG
	,	AVG(CAST([REPORTED] AS FLOAT)) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as REPORTED_3D_AVG

	-- Reported per 1000000 over the last seven days including today. Do only report when there are seven days that can be totaled.
	,   CAST(ROUND(SUM(REPORTED) OVER 
			(PARTITION BY MUNICIPALITY_CODE ORDER BY [DATE_OF_STATISTICS] ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
			/(T1.INHABITANTS/1000000.0),0) AS INT) AS REPORTED_PER_MIL_LAST_WEEK
	,   HOSPITALIZED/(T1.[INHABITANTS]/100000.0) AS HOSPITALIZED_PER_100000
	,   LAG ([DATE_OF_STATISTICS] ,6) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY [DATE_OF_STATISTICS] ASC ) WEEK_START
	,   LAG ([DATE_OF_STATISTICS] ,7) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY [DATE_OF_STATISTICS] ASC ) WEEK_START_LAG
	-- 7d average
	,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as HOSPITALIZED_7D_AVG
	,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as HOSPITALIZED_7D_AVG_LAG
	,	AVG(CAST([REPORTED] AS FLOAT)) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as REPORTED_7D_AVG
	,	AVG(CAST([REPORTED] AS FLOAT)) OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as REPORTED_7D_AVG_LAG
	
	FROM BASE_CTE
	LEFT JOIN (
		SELECT GMCODE, GMNAAM FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL]
		WHERE DATE_LAST_INSERTED=(SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL])) AS T
	ON MUNICIPALITY_CODE=T.GMCODE
	LEFT JOIN(
	    SELECT GMCODE, INHABITANTS
	    FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY]
	    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY])
	) AS T1 ON MUNICIPALITY_CODE = T1.GMCODE 
)

,RANK_TABLE AS (
    SELECT
            DATE_OF_STATISTICS
            ,MUNICIPALITY_CODE
            ,GMNAAM
           ,HOSPITALIZED
           ,REPORTED
           ,HOSPITALIZED_3D_AVG
           ,REPORTED_3D_AVG
           ,IIF(WEEK_START IS NULL, NULL, REPORTED_PER_MIL_LAST_WEEK) AS REPORTED_PER_MIL_LAST_WEEK
		   ,HOSPITALIZED_PER_100000
           ,WEEK_START
           ,IIF(WEEK_START IS NULL, NULL, ROUND(HOSPITALIZED_7D_AVG,1)) AS HOSPITALIZED_7D_AVG
           ,IIF(WEEK_START IS NULL, NULL, ROUND(REPORTED_7D_AVG,1)) AS REPORTED_7D_AVG
           ,WEEK_START_LAG
           ,IIF(WEEK_START_LAG IS NULL, NULL, ROUND(HOSPITALIZED_7D_AVG_LAG,1)) AS HOSPITALIZED_7D_AVG_LAG
           ,IIF(WEEK_START_LAG IS NULL, NULL, ROUND(REPORTED_7D_AVG_LAG,1)) AS REPORTED_7D_AVG_LAG
           ,ROW_NUMBER() OVER (PARTITION BY MUNICIPALITY_CODE ORDER BY DATE_OF_STATISTICS DESC) AS RANK_DATE
    FROM SECOND_CTE
)

INSERT INTO VWSDEST.NICE_HOSPITAL_GM(
	DATE_OF_STATISTICS
,	GM_CODE
,	GM_NAME
,	HOSPITALIZED
,	REPORTED
,	HOSPITALIZED_3D_AVG
,	REPORTED_3D_AVG
,	REPORTED_PER_MIL_LAST_WEEK
,	HOSPITALIZED_PER_100000
,	WEEK_START
,   HOSPITALIZED_7D_AVG
,   REPORTED_7D_AVG
,   WEEK_START_LAG
,   HOSPITALIZED_7D_AVG_LAG
,   REPORTED_7D_AVG_LAG
,   HOSPITALIZED_7D_AVG_CUTOFF
    )

SELECT
        DATE_OF_STATISTICS
       ,MUNICIPALITY_CODE
       ,GMNAAM
       ,HOSPITALIZED
       ,REPORTED
       ,HOSPITALIZED_3D_AVG
       ,REPORTED_3D_AVG
       ,REPORTED_PER_MIL_LAST_WEEK
	   ,HOSPITALIZED_PER_100000
       ,WEEK_START
       ,HOSPITALIZED_7D_AVG
       ,REPORTED_7D_AVG
       ,WEEK_START_LAG
       ,HOSPITALIZED_7D_AVG_LAG
       ,REPORTED_7D_AVG_LAG
       ,IIF(RANK_DATE>4,HOSPITALIZED_7D_AVG,NULL) AS HOSPITALIZED_7D_AVG_CUTOFF
FROM RANK_TABLE
END

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER PROCEDURE [dbo].[SP_NICE_HOSPITAL_VR]
AS
BEGIN
WITH BASE_CTE AS (
	SELECT
		DATE_OF_STATISTICS
	,	SECURITY_REGION_CODE
	,	SUM(HOSPITALIZED) AS HOSPITALIZED
	,	SUM(REPORTED) AS REPORTED
	FROM VWSINTER.NICE_HOSPITAL
	WHERE SECURITY_REGION_CODE!='NA'
	AND DATE_LAST_INSERTED=(SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.NICE_HOSPITAL)
	GROUP BY SECURITY_REGION_CODE, DATE_OF_STATISTICS
)
,
AGEGROUP_POPULATION_DATERANGE AS (
SELECT 
     [VR_CODE]
	,DATUM_PEILING AS DATE_FROM
    -- 2100-01-01 is used as convention for the most recent versions as no end date is known yet.
    ,ISNULL(LEAD(DATUM_PEILING,1) OVER (PARTITION BY [VR_CODE] ORDER BY [DATUM_PEILING] ASC),CAST('2100-01-01' AS DATE)) AS DATE_TO
	,CAST(POPULATIE AS FLOAT) AS [POPULATION]
FROM [VWSSTATIC].[CBS_POPULATION_VR]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_VR)--Determine ranges in which population numbers should be used
) 
,CALCULATED_CTE AS (
	SELECT
		DATE_OF_STATISTICS
	,	SECURITY_REGION_CODE
	,	VRNAAM AS VR_NAME
	,	HOSPITALIZED
	,	REPORTED
	,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as HOSPITALIZED_3D_AVG
	,	AVG(CAST([REPORTED] AS FLOAT)) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as REPORTED_3D_AVG
	,   (SUM(HOSPITALIZED) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC)) AS HOSPITALIZED_CUMULATIVE
	,   (SUM(REPORTED) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC)) AS REPORTED_CUMULATIVE
	,   HOSPITALIZED/(T1.[POPULATION]/100000.0) AS HOSPITALIZED_PER_100000

	-- Reported per 1000000 over the last seven days including today. Do only report when there are seven days that can be totaled.
	,  SUM(REPORTED) OVER  (PARTITION BY SECURITY_REGION_CODE ORDER BY [DATE_OF_STATISTICS] ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
			/(T1.[POPULATION]/1000000.0) AS REPORTED_PER_MIL_LAST_WEEK
	,   LAG ([DATE_OF_STATISTICS] ,6) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY [DATE_OF_STATISTICS] ASC) WEEK_START
	,   LAG ([DATE_OF_STATISTICS] ,7) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY [DATE_OF_STATISTICS] ASC) WEEK_START_LAG
	-- 7d average
	,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as HOSPITALIZED_7D_AVG
	,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as HOSPITALIZED_7D_AVG_LAG
	,	AVG(CAST([REPORTED] AS FLOAT)) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as REPORTED_7D_AVG
	,	AVG(CAST([REPORTED] AS FLOAT)) OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as REPORTED_7D_AVG_LAG

    -- Additional data
    ,T1.[POPULATION]

	FROM BASE_CTE T0

    --Get population at weekstart
    LEFT JOIN AGEGROUP_POPULATION_DATERANGE T1 
        ON  T0.SECURITY_REGION_CODE = T1.[VR_CODE]
        AND (DATEADD(DAY, -6, T0.DATE_OF_STATISTICS)>= T1.DATE_FROM AND DATEADD(DAY, -6, T0.DATE_OF_STATISTICS) < T1.DATE_TO)

    -- Retrieve the correct region name
	LEFT JOIN (
		SELECT VRCODE, VRNAAM FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL]
		WHERE DATE_LAST_INSERTED=(SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL])
		group by VRCODE, VRNAAM
	) AS T2 ON SECURITY_REGION_CODE=T2.VRCODE
)

,RANK_TABLE AS (
    SELECT
        DATE_OF_STATISTICS
        ,SECURITY_REGION_CODE
        ,VR_NAME
        ,HOSPITALIZED
        ,REPORTED
        ,HOSPITALIZED_3D_AVG
        ,REPORTED_3D_AVG
        ,HOSPITALIZED_CUMULATIVE
        ,REPORTED_CUMULATIVE
        ,HOSPITALIZED_PER_100000
           ,IIF(WEEK_START IS NULL, NULL, REPORTED_PER_MIL_LAST_WEEK) AS REPORTED_PER_MIL_LAST_WEEK
           ,WEEK_START
           ,IIF(WEEK_START IS NULL, NULL, ROUND(HOSPITALIZED_7D_AVG,1)) AS HOSPITALIZED_7D_AVG
           ,IIF(WEEK_START IS NULL, NULL, ROUND(REPORTED_7D_AVG,1)) AS REPORTED_7D_AVG
           ,WEEK_START_LAG
           ,IIF(WEEK_START_LAG IS NULL, NULL, ROUND(HOSPITALIZED_7D_AVG_LAG,1)) AS HOSPITALIZED_7D_AVG_LAG
           ,IIF(WEEK_START_LAG IS NULL, NULL, ROUND(REPORTED_7D_AVG_LAG,1)) AS REPORTED_7D_AVG_LAG
           ,ROW_NUMBER() OVER (PARTITION BY SECURITY_REGION_CODE ORDER BY DATE_OF_STATISTICS DESC) AS RANK_DATE
    FROM CALCULATED_CTE
)

INSERT INTO VWSDEST.NICE_HOSPITAL_VR(
	DATE_OF_STATISTICS
,	VR_CODE
,	VR_NAME
,	HOSPITALIZED
,	REPORTED
,	HOSPITALIZED_3D_AVG
,	REPORTED_3D_AVG
,	HOSPITALIZED_CUMULATIVE
,	REPORTED_CUMULATIVE
,   HOSPITALIZED_PER_100000
,	REPORTED_PER_MIL_LAST_WEEK
,	WEEK_START
,   HOSPITALIZED_7D_AVG
,   REPORTED_7D_AVG
,   WEEK_START_LAG
,   HOSPITALIZED_7D_AVG_LAG
,   REPORTED_7D_AVG_LAG
,   HOSPITALIZED_7D_AVG_CUTOFF
)
SELECT
        DATE_OF_STATISTICS
       ,SECURITY_REGION_CODE
       ,VR_NAME
       ,HOSPITALIZED
       ,REPORTED
       ,HOSPITALIZED_3D_AVG
       ,REPORTED_3D_AVG
       ,HOSPITALIZED_CUMULATIVE
       ,REPORTED_CUMULATIVE
       ,HOSPITALIZED_PER_100000
       ,REPORTED_PER_MIL_LAST_WEEK
       ,WEEK_START
       ,HOSPITALIZED_7D_AVG
       ,REPORTED_7D_AVG
       ,WEEK_START_LAG
       ,HOSPITALIZED_7D_AVG_LAG
       ,REPORTED_7D_AVG_LAG
       ,IIF(RANK_DATE>4,HOSPITALIZED_7D_AVG,NULL) AS HOSPITALIZED_7D_AVG_CUTOFF
FROM RANK_TABLE

END

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER PROCEDURE [dbo].[SP_NICE_HOSPITAL_NL]
AS
BEGIN
WITH BASE_CTE AS (
    SELECT
    	DATE_OF_STATISTICS
    ,	SUM(HOSPITALIZED) AS HOSPITALIZED
    ,	SUM(REPORTED) AS REPORTED
    FROM VWSINTER.NICE_HOSPITAL
    WHERE DATE_LAST_INSERTED=(SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.NICE_HOSPITAL)
    GROUP BY DATE_OF_STATISTICS
)

,SECOND_CTE AS (
    SELECT
    	DATE_OF_STATISTICS
    ,	HOSPITALIZED
    ,	REPORTED
    ,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as HOSPITALIZED_3D_AVG
    ,	AVG(CAST([REPORTED] AS FLOAT)) OVER (ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as REPORTED_3D_AVG


    -- Reported per 1000000 over the last seven days including today. Do only report when there are seven days that can be totaled.
    ,   CAST(ROUND(SUM(REPORTED) OVER (ORDER BY [DATE_OF_STATISTICS] ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
            /(T1.INHABITANTS/1000000.0),0) AS INT) AS REPORTED_PER_MIL_LAST_WEEK
    ,   LAG ([DATE_OF_STATISTICS] ,6) OVER (ORDER BY [DATE_OF_STATISTICS] ASC) WEEK_START
    ,   LAG ([DATE_OF_STATISTICS] ,7) OVER (ORDER BY [DATE_OF_STATISTICS] ASC) WEEK_START_LAG    
    
    -- 7d average
    ,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as HOSPITALIZED_7D_AVG
    ,	AVG(CAST([HOSPITALIZED] AS FLOAT)) OVER (ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as HOSPITALIZED_7D_AVG_LAG
    ,	AVG(CAST([REPORTED] AS FLOAT)) OVER (ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as REPORTED_7D_AVG
    ,	AVG(CAST([REPORTED] AS FLOAT)) OVER (ORDER BY DATE_OF_STATISTICS ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as REPORTED_7D_AVG_LAG

    FROM BASE_CTE
    --Retrieve the number of inhabitants
    CROSS JOIN(
            SELECT INHABITANTS FROM [VWSSTATIC].[INHABITANTS_PER_SAFETY_REGION]
            WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTATIC].[INHABITANTS_PER_SAFETY_REGION]) AND VGRNR = 'NL00'
) AS T1 
)

,RANK_TABLE AS (
    SELECT
            DATE_OF_STATISTICS
           ,HOSPITALIZED
           ,REPORTED
           ,HOSPITALIZED_3D_AVG
           ,REPORTED_3D_AVG
           ,IIF(WEEK_START IS NULL, NULL, REPORTED_PER_MIL_LAST_WEEK) AS REPORTED_PER_MIL_LAST_WEEK
           ,WEEK_START
           ,IIF(WEEK_START IS NULL, NULL, ROUND(HOSPITALIZED_7D_AVG,1)) AS HOSPITALIZED_7D_AVG
           ,IIF(WEEK_START IS NULL, NULL, ROUND(REPORTED_7D_AVG,1)) AS REPORTED_7D_AVG
           ,WEEK_START_LAG
           ,IIF(WEEK_START_LAG IS NULL, NULL, ROUND(HOSPITALIZED_7D_AVG_LAG,1)) AS HOSPITALIZED_7D_AVG_LAG
           ,IIF(WEEK_START_LAG IS NULL, NULL, ROUND(REPORTED_7D_AVG_LAG,1)) AS REPORTED_7D_AVG_LAG
           ,ROW_NUMBER() OVER (ORDER BY DATE_OF_STATISTICS DESC) AS RANK_DATE
    FROM SECOND_CTE
)

INSERT INTO VWSDEST.NICE_HOSPITAL_NL(
    DATE_OF_STATISTICS
,	HOSPITALIZED
,	REPORTED
,	HOSPITALIZED_3D_AVG
,	REPORTED_3D_AVG
,   REPORTED_PER_MIL_LAST_WEEK
,   WEEK_START
,   HOSPITALIZED_7D_AVG
,   REPORTED_7D_AVG
,   WEEK_START_LAG
,   HOSPITALIZED_7D_AVG_LAG
,   REPORTED_7D_AVG_LAG
,   HOSPITALIZED_7D_AVG_CUTOFF
    )
    
SELECT 
        DATE_OF_STATISTICS
       ,HOSPITALIZED
       ,REPORTED
       ,HOSPITALIZED_3D_AVG
       ,REPORTED_3D_AVG
       ,REPORTED_PER_MIL_LAST_WEEK
       ,WEEK_START
       ,HOSPITALIZED_7D_AVG
       ,REPORTED_7D_AVG
       ,WEEK_START_LAG
       ,HOSPITALIZED_7D_AVG_LAG
       ,REPORTED_7D_AVG_LAG
       ,IIF(RANK_DATE>4,HOSPITALIZED_7D_AVG,NULL) AS HOSPITALIZED_7D_AVG_CUTOFF
FROM RANK_TABLE
END

### DEST &rarr; DEST MAPPING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER PROCEDURE [dbo].[SP_NICE_HOSPITAL_GM_BASE]
AS
BEGIN
WITH LAST_DATE_OF_REPORT AS (
SELECT
GM_CODE
,   CAST(MAX(DATE_OF_STATISTICS) as datetime) AS [LAST_DATE_OF_REPORT]
FROM VWSDEST.NICE_HOSPITAL_GM
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.NICE_HOSPITAL_GM)
GROUP BY GM_CODE
),
BASE_CTE AS (
SELECT
    DATE_OF_STATISTICS AS DATE_OF_REPORT,
    dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [NEW_DATE_OF_REPORT_UNIX],
    GM_CODE,
    dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_STATISTICS]) AS [OLD_DATE_OF_REPORT_UNIX],
    REPORTED,
	REPORTED_7D_AVG AS [OLD_VALUE],
    DBO.NO_NEGATIVE_NUMBER_F(REPORTED) -
    DBO.NO_NEGATIVE_NUMBER_F(REPORTED_7D_AVG) AS [DIFFERENCE]
FROM VWSDEST.NICE_HOSPITAL_GM
LEFT JOIN(
    SELECT GMCODE, INHABITANTS
    FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY])
) AS T1 ON GM_CODE = T1.GMCODE 
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
FROM VWSDEST.NICE_HOSPITAL_GM)
)
INSERT INTO VWSDEST.NICE_HOSPITAL_GM_BASE
    (DATE_OF_REPORT,
    NEW_DATE_OF_REPORT_UNIX,
    GMCODE,
    OLD_DATE_OF_REPORT_UNIX,
    OLD_VALUE,
    [DIFFERENCE])
SELECT
    DATE_OF_REPORT
    ,NEW_DATE_OF_REPORT_UNIX
    ,T1.GM_CODE AS GMCODE
    ,OLD_DATE_OF_REPORT_UNIX
	,CAST(ROUND(OLD_VALUE, 0) AS INT) AS [OLD_VALUE]
	,REPORTED
		- CAST(ROUND(OLD_VALUE, 0) AS INT) AS [DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
ON T1.[GM_CODE] = T2.[GM_CODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
END

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'NICE_HOSPITAL',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = 'Covid-19 ziekenhuisopnames (volgens NICE registratie) per gemeente per ziekenhuisopnamedatum en meldingsdatum';

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 13 * * MON-FRI', -- AT 02:00 PM, ON monday - friday
    @active = @is_active;

### Sources

In [ ]:
--Selecting all relevant information for sources, processes and dependencies:

--SELECT 
--s.source_content as source, s.name as source_name, s.description as source_description, s.TARGET_name as source_target,
--d2.Name AS process_name, d2.description AS process_description, s.name as process_source_name, 
--dp.Name as dependency_name, d2.name AS dependency_dataflow_name,  d3.name AS dependency_process_name,   dp.Description as dependency_description
-- --,d.*, w.*, p.*, d2.*, s.*
--  FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] d
--  inner join [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] w ON d.Id = w.dataflow_id
--  inner join [DATATINO_ORCHESTRATOR_1].[PROCESSES] p on p.workflow_id = w.Id
--  inner join [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] d2 on p.dataflow_id = d2.id
--  inner join [DATATINO_ORCHESTRATOR_1].[SOURCES] s on p.source_id = s.id
--  left join [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dp on dp.dataflow_id = d2.id
--  left join [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] d3 on dp.dependency_id = d3.id
--
--  where d.name = 'NICE_HOSPITAL'

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50),
        @delimiter_type VARCHAR(50),
        @source_type VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://data.rivm.nl/data/vws/covid-19/COVID-19_ziekenhuisopnames.csv'
    WHEN 'acceptance' THEN 'https://acceptatie.data.rivm.nl/data/vws/covid-19/COVID-19_ziekenhuisopnames.csv'
    ELSE 'datafiles/RIVM/COVID-19_ziekenhuisopnames.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'N/A'
END;

SET @delimiter_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'SemiColon'
    WHEN 'acceptance' THEN 'SemiColon'
    ELSE 'SemiColon'
END;

SET @source_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'CsvFile'
    WHEN 'acceptance' THEN 'CsvFile'
    ELSE 'CsvFile'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_Load_Nice_Hospital';
SET @source_description = 'Load NICE Hospital data into staging';
SET @target_name = 'VWSSTAGE.NICE_HOSPITAL';


SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Version|Date_of_report|Date_of_statistics|Municipality_code|Municipality_name|Security_region_code|Security_region_name|Hospital_admission_notification|Hospital_admission',
    @target_columns = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|MUNICIPALITY_CODE|MUNICIPALITY_NAME|SECURITY_REGION_CODE|SECURITY_REGION_NAME|HOSPITAL_ADMISSION_NOTIFICATION|HOSPITAL_ADMISSION|DATE_LAST_INSERTED=GETDATE',
    @target_name = @target_name,
    @source_type = @source_type,
    @location_type = @location_type,
    @delimiter_type = @delimiter_type,
    @security_profile= @security_profile;


#### Stage to inter

In [ ]:
-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);

SET @source = 'DBO.SP_NICE_HOSPITAL_INTER';
SET @source_name = 'SOURCE_Inter_Nice_Hospital';
SET @source_description = 'Move NICE Hospital data from stage table to inter table';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

#### Inter to dest

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = 'DBO.SP_NICE_HOSPITAL_GM';
SET @source_name = 'SOURCE_Dest_Nice_Hospital_GM';
SET @source_description = 'Move NICE Hospital data from inter table to dest table on municipality level'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = 'DBO.SP_NICE_HOSPITAL_VR';
SET @source_name = 'SOURCE_Dest_Nice_Hospital_VR';
SET @source_description = 'Move NICE Hospital data from inter table to dest table on safety region level'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = 'DBO.SP_NICE_HOSPITAL_NL';
SET @source_name = 'SOURCE_Dest_Nice_Hospital_NL';
SET @source_description = 'Move NICE Hospital data from inter table to dest table on national level'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = 'DBO.SP_NICE_HOSPITAL_GM_BASE';
SET @source_name = 'SOURCE_Dest_Nice_Hospital_GM_BASE';
SET @source_description = 'Move NICE Hospital data from dest table to dest table on GM level'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

### Processes

In [ ]:
-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256),
        @workflow_name NVARCHAR(50) = 'NICE_HOSPITAL';

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_Load_Nice_Hospital';
SET @process_description =  'Load NICE Hospital data into staging';
SET @process_source_name = 'SOURCE_Load_Nice_Hospital';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_name = 'PROCESS_Inter_Nice_Hospital';
SET @process_description =  'Move NICE Hospital data from stage table to inter table';
SET @process_source_name = 'SOURCE_Inter_Nice_Hospital';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
SET @process_name = 'PROCESS_Dest_Nice_Hospital_GM';
SET @process_description =  'Move NICE Hospital data from inter table to dest table on municipality level';
SET @process_source_name = 'SOURCE_Dest_Nice_Hospital_GM';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;


SET @process_name = 'PROCESS_Dest_Nice_Hospital_VR';
SET @process_description =  'Move NICE Hospital data from inter table to dest table on safety region level';
SET @process_source_name = 'SOURCE_Dest_Nice_Hospital_VR';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

SET @process_name = 'PROCESS_Dest_Nice_Hospital_NL';
SET @process_description =  'Move NICE Hospital data from inter table to dest table on national level';
SET @process_source_name = 'SOURCE_Dest_Nice_Hospital_NL';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

SET @process_name = 'PROCESS_Dest_Nice_Hospital_GM_BASE';
SET @process_description =  'Move NICE Hospital data from dest table to dest table on GM level';
SET @process_source_name = 'SOURCE_Dest_Nice_Hospital_GM_BASE';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;



### Dependencies

In [ ]:
-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256),
        @workflow_name NVARCHAR(50) = 'NICE_HOSPITAL';

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_Inter_Nice_Hospital';
SET @dependency_dataflow_name = 'PROCESS_Inter_Nice_Hospital';
SET @dependency_process_name = 'PROCESS_Load_Nice_Hospital';
SET @dependency_description = 'Dependency for PROCESS_Inter_Nice_Hospital';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;




-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_Nice_Hospital_GM';
SET @dependency_dataflow_name ='PROCESS_Dest_Nice_Hospital_GM';
SET @dependency_process_name = 'PROCESS_Inter_Nice_Hospital';
SET @dependency_description = 'Dependency for PROCESS_Dest_Nice_Hospital_GM' ;

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;


-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_Nice_Hospital_VR';
SET @dependency_dataflow_name = 'PROCESS_Dest_Nice_Hospital_VR';
SET @dependency_process_name = 'PROCESS_Inter_Nice_Hospital';
SET @dependency_description = 'Dependency for PROCESS_Dest_Nice_Hospital_VR';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;


-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_Nice_Hospital_NL';
SET @dependency_dataflow_name ='PROCESS_Dest_Nice_Hospital_NL';
SET @dependency_process_name = 'PROCESS_Inter_Nice_Hospital';
SET @dependency_description = 'Dependency for PROCESS_Dest_Nice_Hospital_NL';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;


-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_Nice_Hospital_GM_BASE';
SET @dependency_dataflow_name = 'PROCESS_Dest_Nice_Hospital_GM_BASE';
SET @dependency_process_name = 'PROCESS_Dest_Nice_Hospital_GM';
SET @dependency_description = 'Dependency for PROCESS_Dest_Nice_Hospital_GM_BASE';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO

### PROTOS

#### NL

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

--Use the following query to determine the views in use for protos
--  ;WITH CTE_VIEWS(Result) AS (
--	SELECT '%V_HOSPITAL_ADMISSIONS_GM_LAST_NICE%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_GM_NICE%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_GM_REL_NICE%' UNION All
--	SELECT '%V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_VR%' UNION All
--	SELECT '%V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_MOVING_AVG_PER_REGION_NICE%' UNION All
--	SELECT '%V_DIFFERENCE_RESULTS_PER_REGION__HOSPITAL_TOTAL_COUNTS_PER_REGION_NICE%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_VR_NICE%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_VR_REL_NICE%' UNION All
--	SELECT '%V_RESULTS_PER_REGION%' UNION All
--	SELECT '%V_RESULTS_PER_REGION_NICE%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_VR_LAST_NICE%' UNION All
--	SELECT '%V_DIFFERENCE_INTAKE_HOSPITAL_MA__MOVING_AVERAGE_HOSPITAL_NICE%' UNION All
--	SELECT '%V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_NL%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_NL_NICE%' UNION All
--	SELECT '%V_HOSPITAL_ADMISSIONS_NL_REL_NICE%' UNION All
--	SELECT '%V_DIFFERENCE_HOSPITAL_ADMISSIONS__MOVING_AVERAGE_HOSPITAL_NICE%' UNION All
--	SELECT '%V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_GM%'
--)
--SELECT v.Name as view_name, conf.Name as item_name, v.constraint_value as constraint_value, v.constraint_key_name as constraint_key_name, v.grouped_key_name as grouped_key_name, v.grouped_last_update_name as grouped_last_update_name,
--p.name as proto_name,  conf.columns as columns, conf.layout_type_id as layout_type_id, v.last_update_name as last_update_name, conf.active as is_active,
--V.*, conf.*, p.*
--FROM [DATATINO_PROTO_1].[VIEWS] v 
--INNER JOIN CTE_VIEWS c ON v.Name like c.Result
--inner join [DATATINO_PROTO_1].[CONFIGURATIONS] conf on conf.view_id = v.Id
--inner join [DATATINO_PROTO_1].[PROTOS] p on conf.proto_id = p.Id
--where v.constraint_value = 'GM0400' or v.constraint_value = 'VR01' or v.constraint_value is null
--order by p.name


-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/hospital_nice__admissions_on_date_of_reporting_moving_average',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSIONS_NL_NICE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

#### GM

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSIONS_GM_LAST_NICE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'GM_COLLECTION',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 4, -- VALUESDIRECT
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NICE_HOSPITAL_CHOROPLETH_GM_COLLECTION',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice_choropleth',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'GM_COLLECTION',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 4, -- VALUESDIRECT
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSIONS_GM_NICE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'DATE_UNIX|ADMISSIONS_ON_DATE_OF_REPORTING|ADMISSIONS_ON_DATE_OF_ADMISSION|ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE|ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1, -- LASTVALUE,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NICE_HOSPITAL_CHOROPLETH_GM',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice_choropleth',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'DATE_UNIX|ADMISSIONS_ON_DATE_OF_REPORTING|ADMISSIONS_ON_DATE_OF_ADMISSION|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1, -- LASTVALUE,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_GM',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/hospital_nice__admissions_on_date_of_reporting_moving_average',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id VARCHAR(50) = '3', -- LASTVALUEDIRECT
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

#### VR

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSIONS_VR_LAST_NICE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'VR_COLLECTION',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 4, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NICE_HOSPITAL_CHOROPLETH_VR_COLLECTION',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice_choropleth',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'VR_COLLECTION',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 4, -- VALUESDIRECT
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSIONS_VR_NICE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'DATE_UNIX|ADMISSIONS_ON_DATE_OF_REPORTING|ADMISSIONS_ON_DATE_OF_ADMISSION|ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE|ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED|DATE_OF_INSERTION_UNIX',
        @layout_type_id VARCHAR(50) = '1', -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NICE_HOSPITAL_CHOROPLETH_VR',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice_choropleth',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'DATE_UNIX|ADMISSIONS_ON_DATE_OF_REPORTING|ADMISSIONS_ON_DATE_OF_ADMISSION|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1, -- LASTVALUE,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSIONS_VR_REL_NICE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_nice_sum',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'DATE_START_UNIX|DATE_END_UNIX|ADMISSIONS_PER_1M|DATE_OF_INSERTION_UNIX',
        @layout_type_id VARCHAR(50) = '1',
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_INTAKE_HOSPITAL_REPORTED_VR',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/hospital_nice__admissions_on_date_of_reporting_moving_average',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id VARCHAR(50) = '3',
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO